# Data process 



In [52]:
import pandas as pd
import numpy as np

In [53]:
try:
    df = pd.read_csv('label_RSNA.csv')
    print("File CSV berhasil dibaca.\n")

except FileNotFoundError:
    print("ERROR: File tidak ditemukan. Pastikan nama file dan lokasinya sudah benar.")
    exit()

File CSV berhasil dibaca.



In [54]:
# 2. Pisahkan ID dan LabelType
df[['ImageID', 'LabelType']] = df['ID'].str.rsplit('_', n=1, expand=True)

In [55]:
# 3. Ubah ke format pivot (satu baris per ImageID)
df_pivot = df.pivot_table(index='ImageID', columns='LabelType', values='Label', fill_value=0).reset_index()


In [56]:
# 4. Ambil nama-nama label
label_cols = [c for c in df_pivot.columns if c != 'ImageID']

In [57]:
# 5. Hitung distribusi awal
label_counts = (df_pivot[label_cols] > 0).sum().sort_values(ascending=False)
print("Distribusi awal:\n", label_counts)

Distribusi awal:
 LabelType
any                 107933
subdural             47166
intraparenchymal     36118
subarachnoid         35675
intraventricular     26205
epidural              3145
dtype: int64


In [58]:
# 6. Tentukan target sampling
target_per_label = {label: 10000 for label in label_cols}

In [59]:
# 7. Sampling per label
sampled_ids = set()
for label in label_cols:
    positive_ids = df_pivot[df_pivot[label] > 0]['ImageID']
    n = min(target_per_label[label], len(positive_ids))
    sampled = positive_ids.sample(n=n, random_state=42)
    sampled_ids.update(sampled)

for label in label_cols:
    if label == 'any':
        # ambil data yang any == 0
        target_ids = df_pivot[df_pivot[label] == 0]['ImageID']
    else:
        # ambil data yang labelnya > 0
        target_ids = df_pivot[df_pivot[label] > 0]['ImageID']

    n = min(target_per_label[label], len(target_ids))
    sampled = target_ids.sample(n=n, random_state=42)
    sampled_ids.update(sampled)

In [60]:
# 8. Ambil subset hasil sampling
sampled_df = df_pivot[df_pivot['ImageID'].isin(sampled_ids)]
print(f"Total data hasil sampling: {len(sampled_df)}")

Total data hasil sampling: 55297


In [61]:
label_counts = (sampled_df[label_cols] > 0).sum().sort_values(ascending=False)
print("Distribusi awal:\n", label_counts)

Distribusi awal:
 LabelType
any                 45297
subdural            17827
intraparenchymal    17624
subarachnoid        17002
intraventricular    14561
epidural             3145
dtype: int64


# save

In [62]:
# 9. Simpan ke CSV baru
sampled_df.to_csv('data_55k.csv', index=False)
print("File saved as 'data_20k.csv'")

File saved as 'data_20k.csv'
